In [1]:
import pandas as pd
import numpy as np
import gspread
import statsmodels.formula.api as sm
import statsmodels.api as sma
from google.auth import default
from google.colab import drive
from google.colab import auth
drive.mount('/content/gdrive')
creds, _ = default()
gc = gspread.authorize(creds)
auth.authenticate_user()

Mounted at /content/gdrive


In [2]:
yearList = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]
colList = ["WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF", "LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]
PATH = "/content/gdrive/MyDrive/ZSchool_Projects/Purdue/Fun/Kaggle2023/" #Path to storage of Kaggle provided data
MIN_M, MAX_M = 1101, 1477
MIN_W, MAX_W = 3101, 3477

w = False

In [5]:
if w:
  #worksheet = gc.open('WRegularSeasonDetailedResults').sheet1
  worksheet = gc.open('2023TrainW').sheet1
  rows = worksheet.get_all_values()
  data = pd.DataFrame.from_records(rows)
  minTeam, maxTeam = MIN_W, MAX_W
else:
  #worksheet = gc.open('MRegularSeasonDetailedResults').sheet1
  worksheet = gc.open('2023TrainM').sheet1
  rows = worksheet.get_all_values()
  data = pd.DataFrame.from_records(rows)
  minTeam, maxTeam = MIN_M, MAX_M

data.columns=data.iloc[0]
data = data[1:]

In [4]:
outputYears = []

for year in [2022]:

  outputData = []

  for teamNum in range(minTeam, maxTeam + 1):
    teamData = [0] * len(colList)
    teamCount = 0
    for index in range(1, 1 + len(data["WTeamID"])):
      if data["Season"][index] == str(year) and int(data["WTeamID"][index]) == teamNum:
        teamCount += 1
        metricIndex = 0
        for metric in colList:
          teamData[metricIndex] += float(data[metric][index])
          metricIndex += 1
    if teamCount != 0:
      outputData.append(list((i / teamCount) for i in teamData))
    else:
      outputData.append([-99999] * len(colList))

  outputYears.append(outputData)

  print(year)


2022


In [214]:
worksheet = gc.open('MNCAATourneySeedsReal').sheet1
rows = worksheet.get_all_values()
almostThere = pd.DataFrame.from_records(rows)
almostThere.columns=almostThere.iloc[0]
almostThere = almostThere[1:]
almostResult = []
for item in range(len(almostThere["TeamID"])):
  thisTeam = int(almostThere["TeamID"][item+1])
  almostResult.append(outputYears[0][thisTeam - minTeam])


In [215]:
almostResult
with open(PATH + str(w) + "almost.csv", "w") as file:
  for index1 in range(len(almostResult)):
    for index2 in range(len(almostResult[index1])):
      file.write(f"{almostResult[index1][index2]},")
    file.write(f"\n")

In [216]:
with open(PATH + str(w) + "allData.csv", "w") as file:
  for yearIndex in range(len(outputYears)):
    thisYear = yearList[yearIndex]
    for teamNum in range(0, maxTeam - minTeam):
      file.write(f"{thisYear},{teamNum + minTeam}")
      for metric in outputYears[yearIndex][teamNum]:
        file.write(f",{metric}")
      file.write(f"\n")

In [185]:
with open(PATH + str(w) + "matchData2023_2.csv", "w") as file:
  for index1 in range(1,1 + len(data["WTeamID"])):
    yearIndex = 0 #yearList.index(int(data["Season"][index1]))
    team1 = int(data["Team1"][index1])
    team2 = int(data["Team2"][index1])
    for index2 in range(len(outputYears[yearIndex][team2 - minTeam])):
      file.write(f"{outputYears[yearIndex][team2 - minTeam][index2] - outputYears[yearIndex][team1 - minTeam][index2]},")
    file.write(f"\n")

In [6]:
if w:
  worksheet = gc.open('WRegularSeasonDetailedResults').sheet1
  rows = worksheet.get_all_values()
  data = pd.DataFrame.from_records(rows)
  minTeam, maxTeam = MIN_W, MAX_W
else:
  worksheet = gc.open('MRegularSeasonDetailedResults').sheet1
  rows = worksheet.get_all_values()
  data = pd.DataFrame.from_records(rows)
  minTeam, maxTeam = MIN_M, MAX_M

data.columns=data.iloc[0]
data = data[1:]

In [7]:
y = pd.DataFrame(data["ResultAdj"])
X = data[list((i + "Diff") for i in colList) + ["SeedDiff"]]

#colList = colList[:13]
#X = data[list((i[1:]) for i in colList) + ["SeedDiff"]]

y = pd.to_numeric(y["ResultAdj"])
for col in X:
  X[col] = pd.to_numeric(X[col])

X["ResultAdj"] = y

for colName in (X.columns):
  X = X[np.abs(X[colName]) < 1000]

<ipython-input-7-f933f5dce03b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.to_numeric(X[col])
<ipython-input-7-f933f5dce03b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["ResultAdj"] = y


In [51]:
thisFormula = "ResultAdj ~ SeedDiff "


for colName in X.columns:
  if colName in ["WTODiff", "WDRDiff", "WORDiff", "WStlDiff"]: #["WStlDiff", "WFGMDiff", "LAstDiff", "LTODiff"]: #the variables i end up actually going with
    thisFormula += f"+ {colName} "

print(thisFormula)

mylogit = sm.glm(
    formula = thisFormula, #"ResultAdj ~ SeedDiff + WFGADiff + WFGM3Diff + WFGA3Diff", 
    data = X, family = sma.families.Binomial()
).fit()

mylogit2 = sm.glm(
    formula = thisFormula, #"ResultAdj ~ SeedDiff + WFGADiff + WFGM3Diff + WFGA3Diff", 
    data = X, family = sma.families.Binomial()
).fit_constrained('SeedDiff = 0.05')

mylogit2.params[1] /= 2
print(mylogit.summary())
print(mylogit2.summary())


ResultAdj ~ SeedDiff + WORDiff + WDRDiff + WTODiff + WStlDiff 
                 Generalized Linear Model Regression Results                  
Dep. Variable:              ResultAdj   No. Observations:                 3671
Model:                            GLM   Df Residuals:                     3665
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2056.8
Date:                Thu, 16 Mar 2023   Deviance:                       3388.3
Time:                        14:48:28   Pearson chi2:                 2.71e+03
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1472
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------

In [52]:
predictions = mylogit2.predict()
mylogit == mylogit2

False

In [157]:
predictions[:10]

array([0.7041974 , 0.08327026, 0.83678541, 0.17780914, 0.42046893,
       0.45218047, 0.56942969, 0.39916114, 0.82517199, 0.30027874])

In [53]:
errorList = []

counter = 0
for item in X["ResultAdj"]:
  errorList.append(np.abs(predictions[counter] - item))
  counter += 1

In [54]:
pd.DataFrame(errorList).mean()

0    0.381572
dtype: float64

In [55]:
Xtest = pd.read_csv(PATH + "FalsewarmupTrain.csv")
Xtest.columns = ["Year", "Team1", "Team2", "SeedDiff", "WTODiff", "WDRDiff", "WORDiff", "WStlDiff"]
#Xtest.columns = ["Year", "Team1", "Team2", "SeedDiff", "WFGMDiff", "WStlDiff", "LAstDiff", "LTODiff"]
testPreds = mylogit2.predict(Xtest)

In [63]:
count = 0

with open(PATH + str(w) + "outoutouutoutouwooohooo5.csv", "w") as file:
  file.write("ID,Pred\n")
  for index in range(len(Xtest)):
    year = 2023
    team1 = Xtest["Team1"][index]
    team2 = Xtest["Team2"][index]
    prob = 1 - testPreds[index]
    prob = 0.5 - (2 * (0.5 - prob))
    if prob > 1:
      prob = 1
    if prob < 0:
      prob = 0
    if prob > 0.5 and Xtest["SeedDiff"][index] < 0:
      print(prob, Xtest["SeedDiff"][index])
      count += 1
    file.write(f"{year}_{team1}_{team2},{prob}\n")
  file.write("2023_1101_1102,0.5\n")
  file.write("2023_1101_1103,0.5")

count

0.622144059169063 -10.0
0.6151632059171681 -8.0
0.6243806050525005 -2.0
0.5375336600391156 -9.0
0.7138976130458325 -11.0
0.5366387879759156 -14.0
0.6035226478060958 -3.0
0.5317650750776259 -11.0
0.6434219384256954 -4.0
0.742668751795923 -15.0
0.5463352176291312 -8.0
0.6726888860803943 -12.0
0.523538126312189 -2.0
0.5401514466542805 -13.0
0.6462499843045211 -15.0
0.5197563706218649 -8.0
0.7078062278102006 -12.0
0.6859233413614696 -7.0
0.5842509776030322 -5.0
0.5280808347182449 -13.0
0.662396403055832 -12.0
0.6642846038544961 -5.0
0.6625132237333706 -7.0
0.5727788698492335 -4.0
0.5077994711768197 -6.0
0.6809273128219062 -10.0
0.5178603338682721 -6.0
0.5864316345646365 -13.0
0.7058006309964944 -15.0
0.6732760024831546 -10.0
0.6390115162019012 -10.0
0.5855429450039946 -14.0
0.5791768260308943 -10.0
0.5843006069926135 -4.0
0.546076524468585 -13.0
0.6199368201708892 -15.0
0.5771795599283258 -4.0
0.6509780525982642 -15.0
0.6351585968748501 -5.0
0.617339712633854 -3.0
0.6392037506724986 -6.0
0

1320

In [250]:
print(Xtest["Team1"][len(Xtest)-1])
print(Xtest["Team2"][len(Xtest)-1])
testPreds[len(Xtest)-1]

3476
3477


0.5050907984251892